In [1]:
import numpy as np
from scipy.special import expit
from scipy.special import logsumexp
from scipy.sparse import csr_matrix, hstack
import numpy.testing as npt

In [313]:
test_function()

In [314]:
test_function_negative_inf_values()

In [315]:
test_function_positive_inf_values()

In [316]:
test_function_sparse()

In [317]:
test_gradient()

In [318]:
test_gradient_sparse()

In [319]:
test_numeric_grad()

In [2]:
from modules.losses import BinaryLogisticLoss
from modules.linear_model import LinearModel
from modules.utils import get_numeric_grad

import numpy as np
from scipy.sparse import csr_matrix, hstack
import numpy.testing as npt
import time
import pytest


def test_function():
    loss_function = BinaryLogisticLoss(l2_coef=1.0)
    X = np.array([
        [1, 1, 2],
        [1, 3, 4],
        [1, -5, 6]
    ])
    y = np.array([-1, 1, 1])
    w = np.array([1, 2, 3])
    npt.assert_almost_equal(loss_function.func(X, y, w), 16.00008, decimal=5)

def test_function_negative_inf_values():
    loss_function = BinaryLogisticLoss(l2_coef=0.0)
    X = np.array([
        [1, 10 ** 5],
        [1, -10 ** 5],
        [1, 10 ** 5]
    ])
    y = np.array([1, -1, 1])
    w = np.array([1, 100])
    npt.assert_almost_equal(loss_function.func(X, y, w), 0, decimal=5)

def test_function_positive_inf_values():
    loss_function = BinaryLogisticLoss(l2_coef=0.0)
    X = np.array([
        [1, 10 ** 2],
        [1, -10 ** 2],
        [1, 10 ** 2]
    ])
    y = np.array([-1, 1, -1])
    w = np.array([1, 100])
    npt.assert_almost_equal(loss_function.func(X, y, w), 10000.333334, decimal=5)


def test_gradient():
    loss_function = BinaryLogisticLoss(l2_coef=1.0)
    X = np.array([
        [1, 1, 2],
        [1, 3, 4],
        [1, -5, 6]
    ])
    y = np.array([-1, 1, 1])
    w = np.array([1, 2, 3])
    right_gradient = np.array([0.33325, 4.3335 , 6.66634])
    npt.assert_almost_equal(loss_function.grad(X, y, w), right_gradient, decimal=5)


def test_function_sparse():
    loss_function = BinaryLogisticLoss(l2_coef=1.0)
    X = csr_matrix(np.array([
        [1, 1, 2],
        [1, 3, 4],
        [1, -5, 6]
    ]))
    y = np.array([-1, 1, 1])
    w = np.array([1, 2, 3])
    npt.assert_almost_equal(loss_function.func(X, y, w),  16.000082, decimal=5)


def test_gradient_sparse():
    loss_function = BinaryLogisticLoss(l2_coef=1.0)
    X = csr_matrix(np.array([
        [1, 1, 2],
        [1, 3, 4],
        [1, -5, 6]
    ]))
    y = np.array([-1, 1, 1])
    w = np.array([1, 2, 3])
    right_gradient = np. array([0.33325, 4.3335 , 6.66634])
    npt.assert_almost_equal(loss_function.grad(X, y, w), right_gradient, decimal=5)


def test_numeric_grad():
    result = get_numeric_grad(lambda x: (x ** 2).sum(), np.array([1, 2, 3]), 1e-6)
    npt.assert_almost_equal(result, np.array([2, 4, 6]), decimal=5)


def create_simple_dataset():
    X1 = np.random.randint(1, 4, (1000, 10))
    X2 = np.random.randint(-4, 0, (1000, 10))
    X = csr_matrix(np.vstack((X1, X2)))
    y = np.array([-1] * 1000 + [1] * 1000)
    return X, y


def test_simple_classification_task():
    X, y = create_simple_dataset()
    loss_function = BinaryLogisticLoss(l2_coef=0.1)
    linear_model = LinearModel(
        loss_function=loss_function,
        batch_size=100,
        step_alpha=1,
        step_beta=0,
        tolerance=1e-5,
        max_iter=1000,
    )
    linear_model.fit(X, y)
    predictions = linear_model.predict(X)
    npt.assert_equal(predictions, y)


def test_logging():
    X, y = create_simple_dataset()
    loss_function = BinaryLogisticLoss(l2_coef=0.1)
    linear_model = LinearModel(
        loss_function=loss_function,
        batch_size=None,
        step_alpha=1,
        step_beta=0,
        tolerance=1e-100,
        max_iter=5,
    )
    history = linear_model.fit(X, y, trace=True)
    for key in ['time', 'func']:
        assert key in history
        assert len(history[key]) == 5


@pytest.mark.parametrize("step_alpha, step_beta, answer", [
    (1e-1, 0.5, 0.713865),
    (0.6, 1, 15.134696),
    (0.6, 1.1,  1.436495),
])
def test_full_gd(step_alpha, step_beta, answer):
    X = csr_matrix(np.array([
        [1, 0, 0, 2, 5, 0.9],
        [1, 5, 1, 3, 1, 0.1],
        [1, 0, 0, 2, 1, 0.5],
        [1, 5, 1, 4, 3, 0.32],
        [1, 0, 2, 3, 2, 0.1],
        [1, 5, 2, 5, 4, 0.10],
        [1, 0, 0, 6, 6, 0.28],
        [1, 5, 1, 3, 2, 0.7],
    ]))

    y = np.array([-1, -1, -1, -1, 1, 1, 1, 1])
    w_0 = np.array([0.5, 0.1, 0.3, 0.5, 0.3, 0.5])

    loss_function = BinaryLogisticLoss(l2_coef=5)
    lm = LinearModel(
        loss_function=loss_function,
        step_alpha=step_alpha,
        step_beta=step_beta,
        tolerance=1e-5,
        max_iter=5,
    )
    lm.fit(X, y, w_0=w_0)
    npt.assert_almost_equal(lm.loss_function.func(X, y, lm.get_weights()), answer, decimal=5)


def test_real_sparse_problem():
    data = np.array([1, 1, 1, 1, 1])
    row_ind = np.array([0, 10 ** 3, 10 ** 4, 10 ** 5, 10 ** 6])
    col_ind = np.array([0, 10 ** 4, 10 ** 5, 10 ** 5, 10 ** 6])
    X = csr_matrix((data, (row_ind, col_ind)))
    X = csr_matrix(hstack([csr_matrix(np.ones((X.shape[0], 1))), X]))
    y = np.array([1] * (10 ** 6 + 1))
    y[:5 * 10 ** 5] = -1
    w = np.ones(10 ** 6 + 2)
    loss_function = BinaryLogisticLoss(l2_coef=5)
    start = time.time()
    loss_function.func(X, y, w)
    finish = time.time() - start
    assert finish < 1

In [21]:
tolerance = 0.001
max_iter = 1000
step_alpha = 1
step_beta = 2
batch_size = None
trace = True

In [41]:
w_0 = np.array([0.5,1,3,2,5])

In [65]:
lm = LinearModel(
        loss_function,
        batch_size=1,
        step_alpha=1,
        step_beta=2, 
        tolerance=1e-5,
        max_iter=100,
        random_seed=153,
    )

In [66]:
lm.fit(X, y)

In [67]:
lm.get_objective(X, y)

0.49343904839051844

In [69]:
import numpy as np
from scipy.special import expit
import time


class LinearModel:
    def __init__(
        self,
        loss_function,
        batch_size=None,
        step_alpha=1,
        step_beta=0, 
        tolerance=1e-5,
        max_iter=1000,
        random_seed=153,
        w=None,
        w_val=None,
        **kwargs
    ):
        """
        Parameters
        ----------
        loss_function : BaseLoss inherited instance
            Loss function to use
        batch_size : int
        step_alpha : float
        step_beta : float
            step_alpha and step_beta define the learning rate behaviour
        tolerance : float
            Tolerace for stop criterio.
        max_iter : int
            Max amount of epoches in method.
        """
        self.loss_function = loss_function
        self.batch_size = batch_size
        self.step_alpha = step_alpha
        self.step_beta = step_beta
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.random_seed = random_seed
        self.w = w

    def fit(self, X, y, w_0=None, trace=False, X_val=None, y_val=None):
        """

        Parameters
        ----------
        X : numpy.ndarray or scipy.sparse.csr_matrix
            2d matrix, training set.
        y : numpy.ndarray
            1d vector, target values.
        w_0 : numpy.ndarray
            1d vector in binary classification.
            2d matrix in multiclass classification.
            Initial approximation for SGD method.
        trace : bool
            If True need to calculate metrics on each iteration.
        X_val : numpy.ndarray or scipy.sparse.csr_matrix
            2d matrix, validation set.
        y_val: numpy.ndarray
            1d vector, target values for validation set.

        Returns
        -------
        : dict
            Keys are 'time', 'func', 'func_val'.
            Each key correspond to list of metric values after each training epoch.
        """ 
        import timeit   
        epochs_time = []; epochs_loss = []; epochs_loss_val =[]; history = {} # для сохранения истории обучения
        if w_0 is None:  # задаём начальное значение весов, если оно не задано
            w_0 = np.zeros(X.shape[1])
        k = 0
        diff = 100  # произвольное число, заведомо большее tolerance
        w_k = w_0
        if self.batch_size is None: # обычный градиентный спуск с убывающим темпом обучения
            while (k < self.max_iter) & (diff > self.tolerance):
                if trace:
                    start_time = timeit.default_timer() 
                step_eta = self.step_alpha / ((k+1) ** self.step_beta)
                w_k1 = w_k - step_eta * self.loss_function.grad(X, y, w_k)  # шаг обучения
                diff = w_k1 @ w_k1.T + w_k @ w_k.T - 2 * w_k1 @ w_k.T
                w_k = w_k1
                k += 1               
                if trace:
                    epochs_time.append(timeit.default_timer() - start_time)
                    epochs_loss.append(self.loss_function.func(X, y, w_k))
                    if (X_val is not None) & (y_val is not None):
                        epochs_loss_val.append(self.loss_function.func(X_val, y_val, w_k))
        else: # стохастический градиентный спуск, сначала все признаки перемешиваются, а затем последовательно 
              # выбирается выборка размера batch_size
            import math as m
            number_of_steps = m.ceil(X.shape[0] / self.batch_size)
            while (k < self.max_iter) & (diff > self.tolerance): # под max_iter понимаем количество проходов по всей обуч. выборке
                if trace:
                    start_time = timeit.default_timer()
                np.random.shuffle(X)
                step_eta = self.step_alpha / ((k+1) ** self.step_beta)
                w_epoch_k = w_k
                for i in range(number_of_steps):  # выполняем градиентный спуск для каждого батча 
                    w_epoch_k1 = w_epoch_k - step_eta * self.loss_function.grad(X[self.batch_size*i:self.batch_size*(i+1), :], y[self.batch_size*i:self.batch_size*(i+1)], w_epoch_k)            
                    w_epoch_k = w_epoch_k1
                w_k1 = w_epoch_k
                diff = w_k1 @ w_k1.T + w_k @ w_k.T - 2 * w_k1 @ w_k.T
                w_k = w_k1
                k += 1 
                if trace:
                    epochs_time.append(timeit.default_timer() - start_time)
                    epochs_loss.append(self.loss_function.func(X, y, w_k))
                    if (X_val is not None) & (y_val is not None):
                        epochs_loss_val.append(self.loss_function.func(X_val, y_val, w_k))
        self.w = w_k
        if trace:
            history['time'] = epochs_time
            history['func'] = epochs_loss
            if (X_val is not None) & (y_val is not None):
                 history['func_val'] = epochs_loss_val
            return history
        
      
    def predict(self, X, threshold=0):
        """

        Parameters
        ----------
        X : numpy.ndarray or scipy.sparse.csr_matrix
            2d matrix, test set.
        threshold : float
            Chosen target binarization threshold.

        Returns
        -------
        : numpy.ndarray
            answers on a test set
        """
        return np.sign(expit(X @ self.get_weights()) - threshold) 
        # то есть выдаём 1, если сигмойда превышает значение threshold
        

    def get_optimal_threshold(self, X, y):
        """
        Get optimal target binarization threshold.
        Balanced accuracy metric is used.

        Parameters
        ----------
        X : numpy.ndarray or scipy.sparse.csr_matrix
            2d matrix, validation set.
        y : numpy.ndarray
            1d vector, target values for validation set.

        Returns
        -------
        : float
            Chosen threshold.
        """
        if self.loss_function.is_multiclass_task:
            raise TypeError('optimal threhold procedure is only for binary task')

        weights = self.get_weights()
        scores = X.dot(weights)
        y_to_index = {-1: 0, 1: 1}

        # for each score store real targets that correspond score
        score_to_y = dict()
        score_to_y[min(scores) - 1e-5] = [0, 0]
        for one_score, one_y in zip(scores, y):
            score_to_y.setdefault(one_score, [0, 0])
            score_to_y[one_score][y_to_index[one_y]] += 1

        # ith element of cum_sums is amount of y <= alpha
        scores, y_counts = zip(*sorted(score_to_y.items(), key=lambda x: x[0]))
        cum_sums = np.array(y_counts).cumsum(axis=0)

        # count balanced accuracy for each threshold
        recall_for_negative = cum_sums[:, 0] / cum_sums[-1][0]
        recall_for_positive = 1 - cum_sums[:, 1] / cum_sums[-1][1]
        ba_accuracy_values = 0.5 * (recall_for_positive + recall_for_negative)
        best_score = scores[np.argmax(ba_accuracy_values)]
        return best_score

    def get_weights(self):
        """
        Get model weights

        Returns
        -------
        : numpy.ndarray
            1d vector in binary classification.
            2d matrix in multiclass classification.
            Initial approximation for SGD method.
        """
        return self.w

    def get_objective(self, X, y):
        """
        Get objective.

        Parameters
        ----------
        X : numpy.ndarray or scipy.sparse.csr_matrix
            2d matrix.
        y : numpy.ndarray
            1d vector, target values for X.

        Returns
        -------
        : float
        """
        return self.loss_function.func(X, y, self.get_weights())

In [103]:
def get_numeric_grad(f, x, eps):
	"""
	Function to calculate numeric gradient of f function in x.

	Parameters
	----------
	f : callable
	x : numpy.ndarray
		1d array, function argument
	eps : float
		Tolerance

	Returns
	-------
	: numpy.ndarray
		Numeric gradient.
	"""
	# сделаем матрицу, столбцы которой - измененённый по одной компоненте вектор x
	def numeric_grad(a):
		return (f(a) - f(x)) / eps
	return np.apply_along_axis(numeric_grad, 0,
		x.reshape(-1, 1) @ np.ones(x.shape[0]).reshape(1, -1) + eps * np.eye(x.shape[0]))